# Multi Query Retriever

In [56]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.vectorstores import FAISS

In [57]:
doc = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"})
]

In [58]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
vector_store = FAISS.from_documents(
    documents=doc,
    embedding=OpenAIEmbeddings()
)

In [60]:
similarity_retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = {'k':4})

In [61]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
    llm=ChatOpenAI(model="gpt-5-nano")
)

In [68]:
multiquery_retriever

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F30B64B8C0>, search_kwargs={'k': 5}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is\n    to generate 3 different versions of the given user\n    question to retrieve relevant documents from a vector  database.\n    By generating multiple perspectives on the user question,\n    your goal is to help the user overcome some of the limitations\n    of distance-based similarity search. Provide these alternative\n    questions separated by newlines. Original question: {question}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F30BD634D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F30B634170>, root_client=<openai.OpenAI obj

In [63]:
query = "How to improve energy levels and maintain balance?"


In [64]:
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [65]:
for i, doc in enumerate(similarity_results):
    print("*"*50)
    print(f'Result {i+1}')
    print(doc.page_content)
    print("="*50, '\n')

**************************************************
Result 1
Drinking sufficient water throughout the day helps maintain metabolism and energy.

**************************************************
Result 2
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

**************************************************
Result 3
Consuming leafy greens and fruits helps detox the body and improve longevity.

**************************************************
Result 4
The solar energy system in modern homes helps balance electricity demand.



In [66]:
for i, doc in enumerate(multiquery_results):
    print("*"*50)
    print(f'Result {i+1}')
    print(doc.page_content)
    print("="*50, '\n')

**************************************************
Result 1
Drinking sufficient water throughout the day helps maintain metabolism and energy.

**************************************************
Result 2
Deep sleep is crucial for cellular repair and emotional regulation.

**************************************************
Result 3
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

**************************************************
Result 4
Regular walking boosts heart health and can reduce symptoms of depression.

**************************************************
Result 5
Consuming leafy greens and fruits helps detox the body and improve longevity.

**************************************************
Result 6
The solar energy system in modern homes helps balance electricity demand.



For some reason I am getting 6 outputs i check online and find out the k doesn't apply on multiquery_retrievers to put a limit about how many output should be shown as results, as llm generates 3 new querys on the existing user questions and then tries to find the best ans the output gets sorted from given document obj then merge if any duplicates happens to be there after that the result got shown and the numbers can vary. Also could not find why ***Result 2*** being shown as output the llm might confused with its context is what i think 